author: leezeeyee   
date: 2021/4/12  
link: [github](https://github.com/easilylazy/pattern-recognition) 

In [1]:
import numpy as np

1，现有如下9个样本，假设样本（3，0.4）、（1，1）和（3，3）属于第一类，      
样本（2，0.5）、（3，1）和（1，3）属于第二类，    
样本（1，2）、（2，2）和（3，2）属于第三类，     
请自行设计神经网络实现上述样本的分类，希望能通过设计不同的隐含层数、每层的节点数、不同的学习率、不同的激活函数等对实验结果进行讨论。